# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить два задания

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 5 баллов

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 5 баллов

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам.


__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [ ]:
# !pip install wandb
!pip3 install pytorch_lightning torchmetrics

In [ ]:
!pip install wandb

In [ ]:
# Возможно ниже будет не очень работать WandbLogger с первого разу, перезапустите kernel тетрадки
import wandb

wandb.login()

In [4]:
import math
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from pytorch_lightning.loggers import WandbLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy  as Acur
from torchvision.datasets import ImageFolder
from tqdm import tqdm

# You may add any imports you need

In [5]:
def seed_everything(seed):
    # Фискирует максимум сидов.
    # Это понадобится, чтобы сравнение оптимизаторов было корректным
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(123456)

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
!wget https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip?dl=1 -O dataset.zip && unzip -q dataset.zip

--2022-11-01 17:20:49--  https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/33l8lp62rmvtx40/dataset.zip [following]
--2022-11-01 17:20:49--  https://www.dropbox.com/s/dl/33l8lp62rmvtx40/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf30b9af9c28d005682fd728542.dl.dropboxusercontent.com/cd/0/get/Bv_k_HDvzj0J73-2gx7ofannhS63BCCbl0uLIAjd1DUlI8hktHLZH7z22YyCEC3O6VY2mzc8ZXQf6F38zsb5BoFmB6jM7P_pTt8XP0Fa8anrzLwpYihNWJjiRWJ3uLnHzCVVEE7rLjTpAMxxEw14JVdFSsopbnzlekVZbDziklnFbw/file?dl=1# [following]
--2022-11-01 17:20:50--  https://ucf30b9af9c28d005682fd728542.dl.dropboxusercontent.com/cd/0/get/Bv_k_HDvzj0J73-2gx7ofannhS63BCCbl0uLIAjd1DUlI8hktHLZH7z22YyCEC3O6VY2mzc8ZXQf6F38

## Задание 0

### Что поможет сделать на 10 из 10 (одно задание - 5 баллов)

1. Использовать все возможные методы оптимизации и эксперемнтировать с ними.
2. Подбор learning rate. Пример из прошлого семинара как это делать: [Как найти lr](https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html)

```
  trainer = pl.Trainer(accelerator="gpu", max_epochs=2, auto_lr_find=True) 

  trainer.tune(module, train_dataloader, eval_dataloader)

  trainer.fit(module, train_dataloader, eval_dataloader))
```



3. Аугментация данных. [Документация (полезная)](https://pytorch.org/vision/main/transforms.html), а также [библиотека albumentation](https://towardsdatascience.com/getting-started-with-albumentation-winning-deep-learning-image-augmentation-technique-in-pytorch-47aaba0ee3f8)
4. Подбор архитектуры модели. 
5. Можно написать модель руками свою в YourNet, а можно импортировать не предобученную сетку известной архитектуры из модуля torchvision.models. Один из способов как можно сделать: 

  * `torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)`
  * Документация по возможным моделям и как их можно брать: [Документация (полезная)](https://pytorch.org/vision/stable/models.html)
6. Правильно нормализовывать данные при создании, пример [тык, но тут и в целом гайд от и до](https://www.pluralsight.com/guides/image-classification-with-pytorch)
7. Model Checkpointing. Сохраняйте свой прогресс (модели), чтобы когда что-то пойдет не так вы сможете начать с этого места или просто воспроизвести свои результаты модели, которые обучали. 
 * Пример как можно с wandb тут: [Сохраняем лучшие модели в wandb](https://docs.wandb.ai/guides/integrations/lightning)
 * По простому можно так: [Сохраняем модели в pytorch дока](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

### Подготовка данных

In [86]:
train_transform = transforms.Compose(
    [
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.Resize(224)
    ]
)
val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(224)
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


In [87]:
# YOU CAN DEFINE AUGMENTATIONS HERE

train_dataset = ImageFolder('./dataset/dataset/train', transform=train_transform)
val_dataset = ImageFolder('./dataset/dataset/val', transform=val_transform)
#train_dataset = ImageFolder('C:/Users/Крот/Documents/GitHub/Iad_git/sem_3_hw_2/dataset/dataset/train',transform=train_transform)
#val_dataset = ImageFolder('C:/Users/Крот/Documents/GitHub/Iad_git/sem_3_hw_2/dataset/dataset/val', transform=val_transform)
# REPLACE ./dataset/dataset WITH THE FOLDER WHERE YOU DOWNLOADED AND UNZIPPED THE DATASET

train_dataloader = DataLoader(train_dataset, 
    batch_size=12,
    shuffle=True, 
    num_workers=1) # YOUR CODE HERE
val_dataloader = DataLoader(val_dataset, 
    batch_size=16,
    shuffle=False, 
    num_workers=1) # YOUR CODE HERE

In [88]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Посмотрим на картиночки

In [89]:
for batch in train_dataloader:
    images, class_nums = batch
    for i in range(len(images)):
        #plt.imshow(images[i].permute(1, 2, 0))
        print(images[i].size())
        #plt.show()
        #plt.imshow(images[10].permute(1, 2, 0))
        #plt.show()
    break

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


## Задание 1. 

5 баллов
Добейтесь accuracy на валидации не менее 0.44. В этом задании запрещено пользоваться предобученными моделями и ресайзом картинок.


Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались)
4. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

In [12]:
for i in range(2):
    print(i)

0
1


### Модель (или просто импортируйте не предобученную)

Я решил переписать EfficientNet, так как она вроде показывает очень крутые результаты и тратит не так много времени (меньше параметров)

In [90]:
import torch.nn as nn
from math import ceil

In [91]:
class CNNBlock(nn.Module):
    def __init__(self,
                in_channels:int,out_channels:int, kernel_size: int, stride:int, padding:int, groups:int=1
                ):
      super(CNNBlock,self).__init__()
      self.convb = nn.Conv2d(in_channels = in_channels,
                             out_channels= out_channels,
                             kernel_size = kernel_size,
                             stride = stride,
                             padding = padding,
                             groups=groups)
      self.bn = nn.BatchNorm2d(out_channels)
      self.act = nn.SiLU()


    def forward(self,x):
      x = self.convb(x)
      x=self.bn(x)
      return self.act(x)

In [92]:
class SqueeseExcitation(nn.Module):
    def __init__(self, in_ch=3,reduced_dim=3):
      super(SqueeseExcitation,self).__init__()
      self.sq = nn.Sequential(
          nn.AdaptiveAvgPool2d(1), # C x H x W -> C x 1 x 1
          nn.Conv2d(in_ch,reduced_dim,1), 
          nn.SiLU(),
          nn.Conv2d(reduced_dim,in_ch,1)
      )


    def forward(self,x):
      return x*self.sq(x)

In [93]:
class InvertedResBlock(nn.Module):
    def __init__(self,
                in_ch: int, out_ch:int,kernel_size:int,stride: int, padding:int,
                expand_ratio=1, reduction = 4, # squeeze
                survival_prob = 0.8 # stohastic depth
                ):
      super(InvertedResBlock,self).__init__()
      self.survival_prob = survival_prob
      self.use_residual= in_ch == out_ch and stride == 1
      hidden_dim = in_ch*expand_ratio
      self.expand = in_ch != hidden_dim
      reduced_dim = int(in_ch/reduction)

      if self.expand:
        self.expand_conv = CNNBlock(
            in_ch,hidden_dim,kernel_size=3,stride = 1,padding=1
        )
      
      self.conv = nn.Sequential(
          CNNBlock(
              hidden_dim, hidden_dim, kernel_size,
              stride, padding, groups = hidden_dim
                  ),
          SqueeseExcitation(hidden_dim, reduced_dim),
          nn.Conv2d(hidden_dim, out_ch, kernel_size = 1),
          nn.BatchNorm2d(out_ch)
      )


    def stoch_depth(self,x):
      if not self.training:
        return x
        
        binary_tens = torch.rand(x.shape[0],1,1,1,device=x.device) < self.survival_prob
        return torch.div(x, self.survival_prob)*binary_tens


    def forward(self,inputs):
      x = self.expand_conv(inputs) if self.expand else inputs

      if self.use_residual:
        return self.stoch_depth(self.conv(x))
      else: 
        return self.conv(x)

In [94]:
class YourNet(torch.nn.Module):
    def __init__(self,params, architecture,num_classes=200):
        super(YourNet,self).__init__()
        self.model = architecture
        width_factor,depth_factor,drop_rate = self.calculate_factors(params)
        last_channels = ceil(1280*width_factor)
        # -------------------------------------------
        # Архитектура сетки
        # -------------------------------------------
        self.pool1 = nn.AdaptiveAvgPool2d(1)
        self.features  = self.create_features(
            width_factor,depth_factor,last_channels
        )
        self.classifier = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(last_channels,num_classes)
        )
        # -------------------------------------------
        #Вспомогательные параметры и функции
        # -------------------------------------------
        self.loss_func = torch.nn.CrossEntropyLoss()
        self.targets = torch.Tensor()
        self.preds = torch.Tensor()

    def calculate_factors(self, params, a = 1.2,b =1.1):
      phi, drop_rate = params
      depth_factor = a**phi
      width_factor = b**phi
      return width_factor,depth_factor,drop_rate


    def create_features(self,width_factor,depth_factor,last_channels):
      channels = int(32*width_factor)
      features = [CNNBlock(3,channels,3, stride = 2, padding=1)]
      in_channels = channels

      for expand_ratio, channels, repeats,stride, kernel_size in self.model:
        #print(expand_ratio)
        out_channels = 4*ceil(int(channels*width_factor)/4)
        layers_repeats = ceil(repeats*depth_factor)
        
        for layer in range(layers_repeats):
          features.append(
              InvertedResBlock(
                  in_channels, out_channels, expand_ratio = expand_ratio,
                  stride = stride if layer == 0 else 1,
                  kernel_size = kernel_size,
                  padding = kernel_size//2 # if k = 1; pad = 0, k = 3, pad =1, k = 5, pad =2
              )
          )
          in_channels = out_channels

      features.append(
          CNNBlock(in_channels,last_channels,kernel_size=1,
                   stride = 1, padding=0)
      )
      return nn.Sequential(*features)


    def _forward(self, x):
        x= self.pool1(self.features(x))
        return self.classifier(x.view(x.shape[0],-1))
    
    def forward(self, images, target=None):
        output = self._forward(images)
        # get accuracy score and save it to self.accuracy
        if target is not None:
            loss = self.loss_func(output, target)

            self.targets = torch.cat((self.targets, target.cpu()), 0)
            pred = torch.argmax(output, dim=-1)
            self.preds = torch.cat((self.preds, pred.cpu()), 0)
            self.accuracy = Acur(self.preds.to(torch.int64), self.targets.to(torch.int64))

        return loss
    
        
    def get_accuracy(self, reset=False):
        # return accuracy by all values till now
        if reset:
            self.targets = torch.Tensor()
            self.preds = torch.Tensor() 
        return self.accuracy

### Тренировочный класс lightning

In [95]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.model = model
        self.lr = learning_rate
        pass

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        images, target = train_batch
        loss = self.model(images, target)
        self.log("train_loss", loss, prog_bar=True)  # сохраняет логи в папку, но можно несложно добавить wandb
        self.log("accuracy_train",model.get_accuracy(),prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        images, target = val_batch
        loss = self.model(images, target)
        self.log("val_loss", loss, prog_bar=True)
        self.log("accuracy_val",model.get_accuracy(),prog_bar=True)

In [96]:
lr = 1e-3
architecture = [
    [1,16,1,1,3],
    [6,24,2,2,3],
    [6,40,2,2,5],
    [6,80,3,2,3],
    [6,112,3,1,5],
    [6,192,4,2,5],
    [6,320,1,1,3]
]

params  = [0,0.2]

In [97]:
 # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning

model = YourNet(params,architecture,200).to(device) # YOUR CODE HERE
module = YourModule(
model,
learning_rate=lr ) # YOUR CODE HERE

In [22]:
wandb_logger = WandbLogger(log_model='all')

wandb: Currently logged in as: krot. Use `wandb login --relogin` to force relogin


In [51]:
!pip install -Uqq ipdb
import ipdb

     |████████████████████████████████| 793 kB 6.5 MB/s 
     |████████████████████████████████| 1.6 MB 51.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 7.34.0 which is incompatible.


Automatic pdb calling has been turned OFF


In [99]:
# trainer = pl.Trainer(logger=wandb_logger)  # YOUR CODE HERE
# trainer = pl.Trainer(
#     accelerator="cpu",
#     max_epochs=2
# )
# trainer.fit(module, train_dataloader, val_dataloader)
# 3.5) we can also find best learning rate like this: https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html
trainer = pl.Trainer(logger=wandb_logger,accelerator="cpu", max_epochs=3, auto_lr_find=True)    
trainer.tune(module, train_dataloader, val_dataloader)
trainer.fit(module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/setup.py:178: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

TypeError: ignored

### Валидация результатов задания

In [ ]:
def evaluate_task(model, test_dataloader):
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)
    model.eval()
    accuracy = 0.0
    for images, labels in tqdm(test_dataloader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            loss = model(images, labels)
            acc_batch = model.get_accuracy(reset=False)
        accuracy += acc_batch
    accuracy = accuracy / len(test_dataloader)
    return accuracy

In [ ]:
model = model

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * accuracy / 0.44, 0, 10):.2f} баллов")

In [ ]:
print(accuracy)

In [ ]:
model.get_accuracy(reset=True)

## Задание 2

5 баллов
Добейтесь accuracy на валидации не менее 0.84. В этом задании делать ресайз и использовать претрейн можно.

Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались или можете пойти дальше).
4. Попробуйте сначала посмотреть качество исходной модели без дообучения, сохраните как baseline. Отсюда поймете какие слои нужно дообучать.
5. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

### Модель (или просто импортируйте предобученную)

In [ ]:
class YourNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        pass

    def _forward(self, x):
        # runs the Neural Network
        # YOUR CODE HERE
        pass

    def forward(self, images, target=None):
        # YOUR CODE HERE
        pass

    def get_accuracy(self, reset=False):
        # YOUR CODE HERE
        pass

### Тренировочный класс lightning

In [ ]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        # YOUR CODE HERE
        pass

    def forward(self, x):
        # YOUR CODE HERE
        pass

    def configure_optimizers(self):
        # YOUR CODE HERE
        pass

    def training_step(self, train_batch, batch_idx):
        # YOUR CODE HERE
        pass

    def validation_step(self, val_batch, batch_idx):
        # YOUR CODE HERE
        pass

In [ ]:
wandb_logger = WandbLogger(log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning
device =  # YOUR CODE HERE

model = YourNet().to() # YOUR CODE HERE
module = YourModule() # YOUR CODE HERE

In [ ]:
trainer = pl.Trainer(logger=wandb_logger)  # YOUR CODE HERE
trainer.fit()  # YOUR CODE HERE

### Валидация результатов задания

In [ ]:
model = # Подгрузить свою сохраненную модель сюда

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * (accuracy - 0.5) / 0.34, 0, 10):.2f} баллов")

# Отчёт об экспериментах 

текст писать тут (или ссылочку на wandb/любой трекер экспреиментов) для каждого задания, то есть не обязательно именно тут рисовать графики, если вы используете готовые трекеры/мониторинги ваших моделей.